In [4]:
#Daum으로부터의 데이터 추출 코드, 네이버와 query 형식이 달라 조금 수정

import requests
import datetime
import csv
from bs4 import BeautifulSoup
import time


def news_Finder(search_Sentence,start,end,file_Name):
    
    '''
    (str,datetime.date,datetime.date(or datetime.datetime),str) ---> str
    This function shows the number of news of particular date from start to end.
    
    '''
    
    #end에서 하루를 더 더해 반복문에서 원래 now 일자까지의 기사 개수가 나오도록 조정함.
    end += datetime.timedelta(days=1)

    #20XX00**로 형태를 조정하는 이유: daum에서 리퀘스트 할때 이 형태여야 하기 때문이다.
    end_Date = end.strftime('%Y%m%d')

    #기사 개수를 보길 원하는 시작 날짜을 20XX00** 형태로 나타냄.
    start_Date = start.strftime('%Y%m%d')
    
    #엑셀 차트로 표현하기 위해서 csv 모듈 이용해서 파일 작성
    file = open('Data/Daum/'+file_Name+'.csv','a', encoding='euc-kr',newline='')
    csvWriter = csv.writer(file)
    csvWriter.writerow(['검색어','검색 날짜','뉴스 개수'])
    

    while start_Date != end_Date:
        
        #어떤 날짜인지 표시
        print(start.strftime('%Y.%m.%d'))

        #다음에 리퀘스트를 요청할 날짜를 search_Date로 설정함.
        search_Date = start_Date


        #다음에서 특정 일자의 특정 단어가 들어간 기사의 개수를 찾기 위한 URL 양식
        address = 'https://search.daum.net/search?w=news&sort=recency' + '&q={}&sd={}000000&ed={}235959&period=u'.format(search_Sentence,search_Date,search_Date)
        print(address)

        # HTTP GET Request
        
        req = requests.get(address)
                
       
        #html 소스 가져오기
        html = req.text
        
        # BeautifulSoup으로 html소스를 python객체로 변환하기
        # 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시.

        all_news_information = BeautifulSoup(html, 'html.parser')
        
        #Copy selecter에 의해 복사된 태그 사이에 들어가는 정보만 가져옴, 여기서는 뉴스 개수만 가져오기 위함임.
        news_number_info = all_news_information.select('#resultCntArea')


        #뉴스 개수가 존재하지 않아 태그에 해당하는 정보가 존재하지 않을 때 0건이라는 내용 출력
        if not news_number_info:
            
            csvWriter.writerow([search_Sentence,start.strftime('%Y.%m.%d'),'0건'])


        #데이터들 중에서 온전히 뉴스 개수만 가져오기 위한 과정
        else:
            only_news_number = ((((str(news_number_info[0])).split(' / '))[1]).split('<'))[0]
            only_news_number = only_news_number.replace('약 ','').replace('건','').replace(',','')
            
            
            csvWriter.writerow([search_Sentence,start.strftime('%Y.%m.%d'),only_news_number])


        # 한 과정이 끝날때마다 다음 날짜 뉴스로 넘어가기 위함
        start = start + datetime.timedelta(days=1)
        start_Date = start.strftime('%Y%m%d')

In [ ]:
#Daum 사이트에서 데이터 추출

import time
start = datetime.date(2019,1,10)
end = datetime.datetime.now()
search_Sentences = ['장자연','버닝썬']
for search_Sentence in search_Sentences:
    file_Name = search_Sentence + '_Daum'
    news_Finder(search_Sentence,start,end,file_Name)
    time.sleep(1)